In [6]:

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import tensorflow as tf
from pandas import Series
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Bidirectional
from keras.layers import Input
from keras import layers

import tensorflow as tf
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing import sequence


# fix random seed for reproducibility
tf.random.set_seed(7)

In [7]:
df = pd.read_csv(f'../../db/data/merge/secondary/logs_.csv', index_col=0)
df.reset_index(inplace=True, drop=True)

df

,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
0,-0.056357,0.044868,-0.236804,-0.027523,-0.191681,0.090711,-0.187507,-0.277763,-0.046156,-0.119630,...,-0.147436,0.041166,-0.099046,0.009273,0.090472,-0.190166,-0.139295,0.145630,0.228210,-0.048759
1,0.057418,0.116563,-0.162348,-0.068844,-0.031983,0.047432,-0.211379,-0.267380,-0.023087,0.033756,...,-0.180774,-0.117932,-0.052168,0.159828,0.212410,-0.054403,0.069672,0.087273,0.144556,-0.126891
2,-0.206473,0.062703,-0.262812,-0.018366,-0.243434,0.294601,-0.058796,-0.346427,0.000000,-0.215034,...,-0.255167,-0.003746,-0.240322,-0.190002,0.082882,-0.245215,-0.223152,0.033227,0.323370,-0.029303
3,-0.373553,-0.049272,0.125235,-0.404915,-0.733783,0.309505,0.504518,0.201297,0.063483,-0.320355,...,0.300982,-0.251275,-0.561166,0.052171,0.367330,0.544108,-0.307650,0.348187,0.669742,0.846226
4,-0.036356,-0.236578,-0.062598,0.101392,0.012918,-0.200263,-0.070237,0.133973,0.184438,0.191093,...,0.040733,0.270002,0.152371,-0.147307,-0.350272,-0.203694,-0.098087,-0.024836,-0.256038,-0.111437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111091,-0.111104,-0.019367,-0.046219,-0.060675,-0.152940,0.088497,0.060510,-0.024679,0.303152,0.193995,...,0.078071,0.051580,-0.035225,-0.039843,0.053424,0.017658,-0.083248,0.045766,0.133986,0.100968
111092,-0.111104,-0.019367,-0.046219,-0.060675,-0.152940,0.088497,0.060510,-0.024679,0.303152,0.193995,...,0.078071,0.051580,-0.035225,-0.039843,0.053424,0.017658,-0.083248,0.045766,0.133986,0.100968
111093,-0.111104,-0.019367,-0.046219,-0.060675,-0.152940,0.088497,0.060510,-0.024679,0.303152,0.193995,...,0.078071,0.051580,-0.035225,-0.039843,0.053424,0.017658,-0.083248,0.045766,0.133986,0.100968
111094,-0.111104,-0.019367,-0.046219,-0.060675,-0.152940,0.088497,0.060510,-0.024679,0.303152,0.193995,...,0.078071,0.051580,-0.035225,-0.039843,0.053424,0.017658,-0.083248,0.045766,0.133986,0.100968


In [8]:
#convert logs to classification states = ( 1, 0, -1 ) 

def condition(x, threshold):
    if x>threshold:
        return 1
    elif x<-threshold:
        return -1
    else:
        return 0

def binary(x):
    if x>0:
        return 1
    else:
        return -1


df_ = df.copy()

for sym in df.columns:
    df_[sym] = df[sym].map( lambda x: binary(x) )

df_.reset_index(inplace=True, drop=True)


y = df_.to_numpy()

y.shape

(111096, 28)

In [9]:
# load features

df2 = pd.read_csv('../../db/data/merge/tendency/tendency.csv', index_col=0)
df2.reset_index(inplace=True, drop=True)

df2

X = df2.to_numpy()

X.shape

(111096, 76)

In [ ]:
X_train = np.array([X[1000:int(len(X)*0.2)]])
y_train = np.array([y[1000:int(len(y)*0.2)]])

X_train.shape, y_train.shape

np.unique(y_train)

((1, 21219, 76), (1, 21219, 28))

In [ ]:
x n_sequence, 24 x 7, 76

In [11]:
def initialize_model(XX , yy):

    model = Sequential()
    
    model.add(LSTM(50, activation='relu', input_shape=(XX.shape[1], XX.shape[2])))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(50 , activation='relu'))
    model.add(Dense(yy.shape[-1], activation='sigmoid',))


    model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])

    return model


model = initialize_model(X_train , y_train)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 50)                25400     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 28)                1428      
                                                                 
Total params: 29,378
Trainable params: 29,378
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
1/1 [==============================] - 9s 9s/step - loss: 1.3002 - accuracy: 0.0730
Epoch 2/100
1/1 [==============================] - 8s 8s/step - loss: 1.3002 - accuracy: 0.0730
Epoch 3/100
1/1 [==============================] - 8s 8s/step - loss: 1.3002 - accuracy: 0.0730
Epoch 4/100
1/1 [==============================] - 7s 7s/step - loss: 1.3002 - accuracy: 0.0730
Epoch 5/100
1/1 [==============================] - 8s 8s/step - loss: 1.3002 - accuracy: 0.0730
Epoch 6/100
1/1 [==============================] - 8s 8s/step - loss: 1.3002 - accuracy: 0.0730
Epoch 7/100
1/1 [==============================] - 8s 8s/step - loss: 1.3002 - accuracy: 0.0730
Epoch 8/100
1/1 [==============================] - 8s 8s/step - loss: 1.3002 - accuracy: 0.0730
Epoch 9/100
1/1 [==============================] - 8s 8s/step - loss: 1.3002 - accuracy: 0.0730
Epoch 10/100
1/1 [==============================] - 8s 8s/step - loss: 1.3002 - accuracy: 0.0730
Epoch 11/100
1/1 [=====================

KeyboardInterrupt: 

In [4]:
from tensorflow.keras.layers import Normalization
from tensorflow.keras import layers, Sequential
from tensorflow.keras.optimizers import RMSprop

df = pd.DataFrame(  {'col1': np.arange(0,100), 'col2': np.arange(0,100)  } )
y = np.random.randint(0,2, 100)
y = np.vstack((y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y)).T
y.shape
X_final = np.array([df])
y_final = np.array([y])
X_final.shape, y_final.shape

def create_model_manystocks(X_train, y_train):
    print(X_train.shape, y_train.shape)
    print(X_train.shape[1],X_train.shape[2])
#     tf.random.set_seed(30)
    # normalizer = Normalization() # Instantiate a "normalizer" layer
    # normalizer.adapt(X_train) # "Fit" it on the train set
    model = Sequential()
    # model.add(normalizer)
    model.add(layers.LSTM(40, return_sequences=False, input_shape=(X_train.shape[1],X_train.shape[2])))
#     model.add(layers.LSTM(40, return_sequences=False, recurrent_dropout=0.3))
    # model.add(layers.LSTM(5))
#     model.add(layers.Dropout(0.2))
    model.add(layers.Dense(20, activation='relu'))
    model.add(layers.Dropout(0.2))
    # model.add(layers.Dense(y_train.shape[1], activation='relu'))
    # model.add(layers.Dense(y_train.shape[-1], activation='linear'))
    model.add(layers.Dense(y_train.shape[-1], activation='sigmoid'))
    # model.add(layers.Dense(1, activation='sigmoid'))
    # model.compile(loss='mse',optimizer=RMSprop(learning_rate=0.008), metrics=['mae', 'mape'])
    model.compile(loss='mse',optimizer=RMSprop(learning_rate=0.008), metrics=['accuracy'])
    # model.compile(loss='mse',optimizer='rmsprop', metrics=['mae', 'mape'])
    print(model.summary())
    print(X_train.shape)
    return model

model = create_model_manystocks(X_final, y_final)


(1, 100, 2) (1, 100, 28)
100 2
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 40)                6880      
                                                                 
 dense (Dense)               (None, 20)                820       
                                                                 
 dropout (Dropout)           (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 28)                588       
                                                                 
Total params: 8,288
Trainable params: 8,288
Non-trainable params: 0
_________________________________________________________________
None
(1, 100, 2)


In [5]:
model.fit(X_final, y_final, epochs=10)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.2596 - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 28ms/step - loss: 0.2654 - accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 28ms/step - loss: 0.2620 - accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 27ms/step - loss: 0.2503 - accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 28ms/step - loss: 0.2485 - accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 29ms/step - loss: 0.2485 - accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 26ms/step - loss: 0.2484 - accuracy: 0.0000e+00
Epoch 8/10
1/1 [==============================] - 0s 28ms/step - loss: 0.2479 - accuracy: 0.0000e+00
Epoch 9/10
1/1 [==============================] - 0s 29ms/step - loss: 0.2477 - accuracy: 0.0000e+00
Epoch 10/10
1/1 [==============================] - 0s 26ms/step - loss: 0.2477 - accuracy: 0.

In [64]:
! pip freeze | grep tensorflow

tensorflow==2.9.1
tensorflow-estimator==2.9.0
tensorflow-io-gcs-filesystem==0.26.0
